In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp -av '/content/drive/MyDrive/inteligencia2/segundoParcial/cuentos.txt' '/content/data.txt'

'/content/drive/MyDrive/inteligencia2/segundoParcial/cuentos.txt' -> '/content/data.txt'


In [3]:
f = open("data.txt", "r", encoding='utf-8')
text = f.read()
text[:300], len(text)

('El desayuno de Laura\nA las ocho de la mañana la mamá de Laura ya se ha tomado su café con tostadas. Es hora de despertar a su hija o se hará tarde. Casi a oscuras, se acerca a la pequeña cama de madera y busca su carita bajo el edredón para darle un beso de buenos días.\nLaura se despereza, se pone s',
 80375)

In [17]:
import string

all_characters = '0123456789abcdefghijlmnopqrstuvyABCDEFGHIJLMNOPRSTUVY \n'
all_characters

'0123456789abcdefghijlmnopqrstuvyABCDEFGHIJLMNOPRSTUVY \n'

In [18]:
import string

all_characters = string.printable + "ñÑáÁéÉíÍóÓúÚ¿¡"
all_characters

'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0cñÑáÁéÉíÍóÓúÚ¿¡'

In [19]:
import string

class Tokenizer(): 
    
  def __init__(self):
    self.all_characters = all_characters
    self.n_characters = len(self.all_characters)
    
  def text_to_seq(self, string):
    seq = []
    for c in range(len(string)):
        try:
            seq.append(self.all_characters.index(string[c]))
        except:
            continue
    return seq

  def seq_to_text(self, seq):
    text = ''
    for c in range(len(seq)):
        text += self.all_characters[seq[c]]
    return text

tokenizer = Tokenizer()
tokenizer.n_characters

114

In [20]:
tokenizer.text_to_seq('fin')

[15, 18, 23]

In [21]:
tokenizer.seq_to_text([15, 18, 23])

'fin'

In [22]:
text_encoded = tokenizer.text_to_seq(text)

In [23]:
train_size = len(text_encoded) * 80 // 100 
train = text_encoded[:train_size]
test = text_encoded[train_size:]

len(train), len(test)

(64167, 16042)

In [24]:
import random

def windows(text, window_size = 100):
    start_index = 0
    end_index = len(text) - window_size
    text_windows = []
    while start_index < end_index:
      text_windows.append(text[start_index:start_index+window_size+1])
      start_index += 1
    return text_windows

text_encoded_windows = windows(text_encoded)

In [25]:
print(tokenizer.seq_to_text((text_encoded_windows[0])))
print()
print(tokenizer.seq_to_text((text_encoded_windows[1])))
print()
print(tokenizer.seq_to_text((text_encoded_windows[2])))

El desayuno de Laura
A las ocho de la mañana la mamá de Laura ya se ha tomado su café con tostadas. E

l desayuno de Laura
A las ocho de la mañana la mamá de Laura ya se ha tomado su café con tostadas. Es

 desayuno de Laura
A las ocho de la mañana la mamá de Laura ya se ha tomado su café con tostadas. Es 


In [26]:
import torch

class CharRNNDataset(torch.utils.data.Dataset):
  def __init__(self, text_encoded_windows, train=True):
    self.text = text_encoded_windows
    self.train = train

  def __len__(self):
    return len(self.text)

  def __getitem__(self, ix):
    if self.train:
      return torch.tensor(self.text[ix][:-1]), torch.tensor(self.text[ix][-1])
    return torch.tensor(self.text[ix])

In [27]:
train_text_encoded_windows = windows(train)
test_text_encoded_windows = windows(test)

dataset = {
    'train': CharRNNDataset(train_text_encoded_windows),
    'val': CharRNNDataset(test_text_encoded_windows)
}

dataloader = {
    'train': torch.utils.data.DataLoader(dataset['train'], batch_size=512, shuffle=True, pin_memory=True),
    'val': torch.utils.data.DataLoader(dataset['val'], batch_size=2048, shuffle=False, pin_memory=True),
}

len(dataset['train']), len(dataset['val'])

(64067, 15942)

In [29]:
input, output = dataset['train'][0]
tokenizer.seq_to_text(input)

'El desayuno de Laura\nA las ocho de la mañana la mamá de Laura ya se ha tomado su café con tostadas. '

In [30]:
tokenizer.seq_to_text([output])

'E'

In [31]:
class CharRNN(torch.nn.Module):
  def __init__(self, input_size, embedding_size=128, hidden_size=256, num_layers=3, dropout=0.2):
    super().__init__()
    self.encoder = torch.nn.Embedding(input_size, embedding_size)
    self.rnn = torch.nn.LSTM(input_size=embedding_size, hidden_size=hidden_size, num_layers=num_layers, dropout=dropout, batch_first=True)
    self.fc = torch.nn.Linear(hidden_size, input_size)

  def forward(self, x):
    x = self.encoder(x)
    x, h = self.rnn(x)         
    y = self.fc(x[:,-1,:])
    return y

In [32]:
model = CharRNN(input_size=tokenizer.n_characters)
outputs = model(torch.randint(0, tokenizer.n_characters, (64, 50)))
outputs.shape

torch.Size([64, 114])

In [33]:
from tqdm import tqdm
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"

def fit(model, dataloader, epochs=10):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = torch.nn.CrossEntropyLoss()
    for epoch in range(1, epochs+1):
        model.train()
        train_loss = []
        bar = tqdm(dataloader['train'])
        for batch in bar:
            X, y = batch
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            y_hat = model(X)
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()
            train_loss.append(loss.item())
            bar.set_description(f"loss {np.mean(train_loss):.5f}")
        bar = tqdm(dataloader['val'])
        val_loss = []
        model.eval()
        with torch.no_grad():
            for batch in bar:
                X, y = batch
                X, y = X.to(device), y.to(device)
                y_hat = model(X)
                loss = criterion(y_hat, y)
                val_loss.append(loss.item())
                bar.set_description(f"val_loss {np.mean(val_loss):.5f}")
        print(f"Epoch {epoch}/{epochs} loss {np.mean(train_loss):.5f} val_loss {np.mean(val_loss):.5f}")

def predict(model, X):
    model.eval() 
    with torch.no_grad():
        X = torch.tensor(X).to(device)
        pred = model(X.unsqueeze(0))
        return pred

In [34]:
model = CharRNN(input_size=tokenizer.n_characters)
fit(model, dataloader, epochs=30)

val_loss 3.14572: 100%|██████████| 8/8 [00:01<00:00,  5.28it/s]


Epoch 1/30 loss 3.12809 val_loss 3.14572


val_loss 2.56107: 100%|██████████| 8/8 [00:01<00:00,  5.22it/s]


Epoch 2/30 loss 2.78741 val_loss 2.56107


val_loss 2.40942: 100%|██████████| 8/8 [00:01<00:00,  5.07it/s]


...

Epoch 28/30 loss 1.30651 val_loss 1.81741


val_loss 1.82111: 100%|██████████| 8/8 [00:01<00:00,  4.85it/s]


Epoch 29/30 loss 1.28990 val_loss 1.82111


val_loss 1.81839: 100%|██████████| 8/8 [00:01<00:00,  4.41it/s]

Epoch 30/30 loss 1.26859 val_loss 1.81839


In [35]:
X_new = "eraze una vez"
X_new_encoded = tokenizer.text_to_seq(X_new)
y_pred = predict(model, X_new_encoded)
y_pred = torch.argmax(y_pred, axis=1)[0].item()
tokenizer.seq_to_text([y_pred])

' '

In [36]:
temp=1
for i in range(1000):
  X_new_encoded = tokenizer.text_to_seq(X_new[-100:])
  y_pred = predict(model, X_new_encoded)
  y_pred = y_pred.view(-1).div(temp).exp()
  top_i = torch.multinomial(y_pred, 1)[0]
  predicted_char = tokenizer.all_characters[top_i]
  X_new += predicted_char

print(X_new)

eraze una vez candero del puercoza de tu mentido días y la preparablen opes.
Dendre de sus casa duerde quíullaso
Si oz, gusta y tanto que se todas la madres que sea que llegar y se puso desearmir y ciullás todo y recipionado, al obre llamado, llego con fuerte! Cuando el cuento al Avilno Cerito, profundo que se muñve. Si paba se llono había citaban tan conersente, un papa tracias que los pojor una noble al de que nos lo expertente entanla a la respondió en la grampaga Naparita a béczar le había quería comparten tan ayuda.
Yo este píbnto si puedes cultidad de ocalor, hermosa día tiempo, prajumtar a su solorín conajo y Pero no era una partipa, peraos y sabes el tosques a la lleto para fueras y dos muy muchos porque nos alomajos en su papa de transfordad de su salva, pero ban su paboque, y pues Mabrando.
Madora se mucho cosas porque es caquitor fuel y pompor callé: Si cabajado es muy buen, para cieron muy muy conperterana, los zapatillas en la jufaba peraomrocha, se nuego es una calambrida